# Enquête Ménages Déplacements (EMD), Lyon / Aire métropolitaine lyonnée - 2015
EDA on 2015 [EMD Lyon transportation survey](https://data.progedo.fr/studies/doi/10.13144/lil-1023).

Try reading `Questionnaire_EDGT_FaF_Lyon_2015.pdf` with the **correctly formatted questions** using `pdfplumber`.

Next, get variable names from `Dessin_fichier_Dictionnaire_variables_EDGT_AML_Face-a-Face_02082015.xls`

In [69]:
import pandas as pd
from typing import Tuple, List

config_folder = "../configs/Lyon"
file_path = "../configs/Lyon/data/Dessin_fichier_Dictionnaire_variables_EDGT_AML_Face-a-Face_02082015.xls"

def get_lyon_vars(file_path: str) -> Tuple[List[str]]:
    """Returns the encoded variable names of the Lyon survey for
            1. Menage = household
            2. Personn = trip purpose
            3. Deplacement = trip properties
            4. Opinion = opinion

    Args:
        file_path (str): path to Lyon variable dictionary .xls file

    Returns:
        tuple(List[str]): tuple of variable names for survey parts
    """
    df = pd.read_excel(file_path, sheet_name=0, header=1)
    exclude_vars = ["Variables", "MP1", "ZFM", "ECH", "DATE"]
    variables = df.Variables.dropna()[~df.Variables.isin(exclude_vars)]

    menage_vars = [var for var in variables if var.startswith("M")]         # MX
    personne_vars = [var for var in variables if var.startswith("P")]       # PX
    deplacement_vars = [var for var in variables if var.startswith("D")]    # DX
    trajet_vars = [var for var in variables if var.startswith("T")]         # TX
    opinion_vars = [var for var in variables if var.startswith("O")]        # OX

    return menage_vars, personne_vars, deplacement_vars, trajet_vars, opinion_vars


lyon_variables = get_lyon_vars(file_path)

def get_fiche_responses(excel_path: str):
    # read df, get var name series and var response df
    df = pd.read_excel(excel_path, sheet_name=1)

    # get list of list of variables that corresponds to grouped response options
    var_series = df.iloc[:,0]
    var_series = var_series[~var_series.str.contains("FILTRE", na=False)]

    # group on 'islands' of vars
    mask =var_series.notna()
    groups = (mask != mask.shift()).cumsum()
    var_groups = [group.to_list() for key, group in var_series.groupby(groups) if group.notna().any()]

    # concat and convert groups of vars to one list
    formatted_var_groups = []
    for group in var_groups:
        if len(group) == 1 and not "," in group[0]:
            formatted_var_groups.append(group)
        elif len(group) == 1:
            formatted_group = [var.strip() for var in group[0].split(",")]
            formatted_var_groups.append(formatted_group)
        elif len(group) >= 1:
            formatted_vars = []
            for subgroup in group:
                formatted_subgroup = [var.strip() for var in subgroup.split(",")]
                formatted_vars.extend(formatted_subgroup)
            formatted_var_groups.append(formatted_vars)
        else:
            print(f"Group {group} not formatted")

    # group response options - if correct should be the same
    # length as formatted_var_groups
    response_options_df = df.iloc[:,1:]

    # get mask and index for each group
    mask = response_options_df.iloc[:,1].notna()
    group_id = (mask != mask.shift()).cumsum()
    filtered_groups = response_options_df[mask].groupby(group_id)
    chunks = [group for _, group in filtered_groups]

    return formatted_var_groups, chunks

def df_to_dict(df: pd.DataFrame):
    df = df.dropna()
    try:
        converted_dict = dict(zip(df["FICHE MENAGE"].astype(int).astype(str), df["Unnamed: 2"]))
    except:
        converted_dict = dict(zip(df["FICHE MENAGE"].astype(str), df["Unnamed: 2"]))
    return converted_dict

In [95]:
import re
from pathlib import Path

def process_EnqueteMenagesDeplacements(config_folder:str):
    data_path = Path(config_folder) / "data"
    data_dictionary_path = data_path / "Dessin_fichier_Dictionnaire_variables_EDGT_AML_Face-a-Face_02082015.xls"
    questions_path = Path(config_folder) / "questions.csv"

    # questions df
    questions_df = pd.read_csv(questions_path, header=None, names=["var", "question"])

    # variables and responses from data dictionary
    var_groups, chunks = get_fiche_responses(data_dictionary_path)

    formatted_responses = [df_to_dict(chunk) for chunk in chunks]
    question_vars = questions_df.iloc[:,0].to_list()

    query_dictionary = {}
    re_ignore = ["JOURDEP", "M12A"]

    for group_index, var_group in enumerate(var_groups):
        for var in var_group:
            if var not in re_ignore:
                var = re.sub(r'([A-Z]+\d+)(?:[A-Z]$|-\d+$)', r'\1', var)
            if var in question_vars:
                query_dictionary[var] = {
                    "question": questions_df[questions_df["var"] == var]["question"].values[0],
                    "dtype": "TEXT",
                    "response": formatted_responses[group_index]
                }

    return query_dictionary

query_dictionary = process_EnqueteMenagesDeplacements(config_folder=config_folder)

In [96]:
query_dictionary

{'M1': {'question': 'Type d’habitat dans lequel est intégré le logement',
  'dtype': 'TEXT',
  'response': {'1': 'INDIVIDUEL ISOLE',
   '2': 'INDIVIDUEL ACCOLE',
   '3': 'PETIT COLLECTIF (R+1 à R+3)',
   '4': 'GRAND COLLECTIF (R+4 ET PLUS)',
   '5': 'AUTRES'}},
 'M2': {'question': 'Le ménage est-il ?',
  'dtype': 'TEXT',
  'response': {'1': ' PROPRIÉTAIRE OU ACCÉDANT À LA PROPRIÉTÉ',
   '2': ' LOCATAIRE HLM',
   '3': ' AUTRE LOCATAIRE',
   '4': ' LOGÉ GRATUITEMENT',
   '5': 'LOCATAIRE EN RÉSIDENCE UNIVERSITAIRE (UNIQUEMENT POUR L’ÉCHANTILLON SPÉCIFIQUE ÉTUDIANT)',
   '6': ' AUTRES'}},
 'M3': {'question': 'Avez-vous le téléphone fixe au domicile ?',
  'dtype': 'TEXT',
  'response': {'1': 'OUI', '2': 'NON'}},
 'M5': {'question': 'Avez-vous une connexion Internet ?',
  'dtype': 'TEXT',
  'response': {'1': 'OUI', '2': 'NON'}},
 'M4': {'question': 'Si oui, êtes-vous ?',
  'dtype': 'TEXT',
  'response': {'1': ' ABONNÉ FRANCE TÉLÉCOM, SUR LISTE ROUGE',
   '2': ' ABONNÉ FRANCE TÉLÉCOM, SUR LIS

In [88]:

formatted_responses = [df_to_dict(chunk) for chunk in chunks]
question_vars = questions_df.iloc[:,0].to_list()

query_dictionary = {}
re_ignore = ["JOURDEP", "M12A"]

for group_index, var_group in enumerate(var_groups):
    for var in var_group:
        if var not in re_ignore:
            var = re.sub(r'([A-Z]+\d+)(?:[A-Z]$|-\d+$)', r'\1', var)
        else:
            print(var)
        if var in question_vars:
            query_dictionary[var] = {
                "question": questions_df[questions_df["var"] == var]["question"].values[0],
                "response": formatted_responses[group_index]
            }

query_dictionary

M12A
JOURDEP


{'M1': {'question': 'Type d’habitat dans lequel est intégré le logement',
  'response': {'1': 'INDIVIDUEL ISOLE',
   '2': 'INDIVIDUEL ACCOLE',
   '3': 'PETIT COLLECTIF (R+1 à R+3)',
   '4': 'GRAND COLLECTIF (R+4 ET PLUS)',
   '5': 'AUTRES'}},
 'M2': {'question': 'Le ménage est-il ?',
  'response': {'1': ' PROPRIÉTAIRE OU ACCÉDANT À LA PROPRIÉTÉ',
   '2': ' LOCATAIRE HLM',
   '3': ' AUTRE LOCATAIRE',
   '4': ' LOGÉ GRATUITEMENT',
   '5': 'LOCATAIRE EN RÉSIDENCE UNIVERSITAIRE (UNIQUEMENT POUR L’ÉCHANTILLON SPÉCIFIQUE ÉTUDIANT)',
   '6': ' AUTRES'}},
 'M3': {'question': 'Avez-vous le téléphone fixe au domicile ?',
  'response': {'1': 'OUI', '2': 'NON'}},
 'M5': {'question': 'Avez-vous une connexion Internet ?',
  'response': {'1': 'OUI', '2': 'NON'}},
 'M4': {'question': 'Si oui, êtes-vous ?',
  'response': {'1': ' ABONNÉ FRANCE TÉLÉCOM, SUR LISTE ROUGE',
   '2': ' ABONNÉ FRANCE TÉLÉCOM, SUR LISTE ANTI-PROSPECTION (EX LISTE ORANGE)',
   '3': " ABONNÉ FRANCE TÉLÉCOM, SUR L'ANNUAIRE DES ABO

In [71]:
questions_df

,var,question
0,TEST,this is a test
1,M1,Type d’habitat dans lequel est intégré le loge...
2,M2,Le ménage est-il ?
3,M3,Avez-vous le téléphone fixe au domicile ?
4,M4,"Si oui, êtes-vous ?"
...,...,...
111,O31,"En général, quel mode de transport utilisez-vo..."
112,O32a,Avez-vous déjà utilisé « OnlyMoov » ?
113,O33,Pour vous rendre dans les grandes villes de la...
114,O34,Avez-vous déjà utilisé « Multitud.org » pour p...
